### Ingest title_ratings file

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

##### Step 1 - Read .tsv file

In [0]:
title_ratings_raw_df = spark.table("imdb_dev.bronze.title_ratings")

In [0]:
display(title_ratings_raw_df)

##### Step 2 - Rename Columns

In [0]:
from pyspark.sql.functions import col, lit, split, when

In [0]:
title_ratings_raw_df.columns

In [0]:
title_ratings_renamed_df = title_ratings_raw_df \
                        .withColumnRenamed("averageRating", "average_rating") \
                        .withColumnRenamed("numVotes", "num_votes") \
                        .withColumn("data_source", lit(v_data_source))

##### Step 4 - Add ingestion date

In [0]:
title_ratings_final_df = add_ingestion_date(title_ratings_renamed_df)

In [0]:
display(title_ratings_final_df)

##### Step 5 - Write data to datalake as delta table

In [0]:
(
    title_ratings_final_df.write
      .format("delta")
      .mode("overwrite")
      .option("overwriteSchema", "true")                                  
      .saveAsTable("imdb_dev.silver.title_ratings")
)